# Single gene trees

In this notebook, we infer single gene trees with the 94 plastid genes including both the references and the plastid MAGs, with the purpose of identifying contaminants, chimeras, and cases of LGT. 

However, manually parsing trees to identify such cases can be difficult when the trees contain hundreds of unlabelled environmental sequences. Therefore, prior to these phylogenies, we ran a quick-and-dirty set of SGTs with IQTree. Taxa were filtered if they had low coverage (present in fewer than 10 genes), were chimeras, or were bacterial contaminants (i.e. clustered with bacteria in the trees). Following this, a concatenated tree of the 34 "slow-genes" of [Ševčíková et al 2015](https://doi.org/10.1038/srep10134) and [Janouškovec et al 2010](https://doi.org/10.1073/pnas.1003335107) was inferred. The concatenated tree was run in IQTREE under the LG+C40+I+G model. (These preliminary analyses are not described in these notebooks but documentation is available on request). Based on the concatenated tree, we derived the taxonomic position of the MAGs, and labelled the MAGs with the taxonomy, making it easier to parse the next round of single gene trees and spot any aberrant sequences.   

In [ ]:
# Check if python is 3.10.5
import json
import os
import pandas as pd
import sys
import numpy as np
import __init__


print(sys.version)
%load_ext autoreload
%autoreload 2

In [11]:
# we store the important data paths in PATH_FILE
PATH_FILE = "../../PATHS.json"

paths_dict = json.load(open(PATH_FILE, "r"));

## 1. Collect homologs

Step 1 is extracting the 68 (+28) gene sequences from each MAG using BLAST. As query, we will use reference sequences [collected in the previous step](01_refs_SGTs.ipynb). 

### 1.1 Put query (reference) sequences in working_dataset folder

In [ ]:
## Directory containing reference sequences
REF = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF']['FINAL']

## Output folder where we copy the reference sequences
WORKING_DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['REF']

In [ ]:
%%bash -s "$FINAL" "$WORKING_DATASET"

cp "$1"/*fasta "$2"/.

### 1.2 Create blast databases for all ptMAG proteomes

In [ ]:
## Folder containing proteomes of the ptMAGs
PROTEOMES = paths_dict['ANALYSIS_DATA']['PLASTOMES']['MFANNOTATIONS']['PROTEOMES']

## Output folder that will contain the blast dbs of the ptMAGs
TO_ADD = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['TOADD']

In [ ]:
%%bash -s "$PROTEOMES" "$TO_ADD"

## load the blast module on Uppmax
module load bioinfo-tools
module load blast

for i in "$1"/*fasta; do
    mag=$(basename $i | cut -f 1 -d '.')
    makeblastdb -in $i -dbtype prot -out "$2"/"$mag".db
done

### 1.3 Blast references against the blast databases

We first create a text file called `blastdb_name.txt` with the names of the blast databases.

In [ ]:
%%bash -s "$TO_ADD"

for i in "$1"/*; do
    echo $i | \
    cut -f 1 -d '.' | \
    sed -E 's/(.*)/\1\.db/'
done | uniq > "$1"/blastdb_name.txt

In [ ]:
## Folder containing the reference fasta files that we will use as blast query
WORKING_DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['REF']

## Folder containing the blast dbs
TO_ADD = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['TOADD']

We submit the blast jobs now!

In [ ]:
%%bash -s "$WORKING_DATASET" "$TO_ADD" 

for i in  "$1"/*.fasta; do
    sbatch ../../uppmax_scripts/script_bin/job_running_blast.sh \
        $i "$2"/blastdb_name.txt
    sleep 1
done

### 1.4 Extract best BLAST hits

Blast output files are generated in the folder containing the reference query sequences. For each gene, we want to:
- parse the blast output files, 
- extract the best hit,
- pull out the corresponding sequence

In [ ]:
## Full gene dataset
JANO_FULL = paths_dict['DATABASES']["GENE_LISTS"]["JANO_FULL"]

# Genes to add
GENES_ADD = paths_dict['DATABASES']["GENE_LISTS"]["GENES_ADD"]

## Folder containing the query fasta files and the blast outputs
WORKING_DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['REF']

## Folder containing proteomes of ptMAGs
PROTEOMES = paths_dict['ANALYSIS_DATA']['PLASTOMES']['MFANNOTATIONS']['PROTEOMES']

We first pull the best blast hit. We can do that by taking the first line of the blast output (since the blast output is already sorted). The second column of the blast tabular output contains the name of the best sequence.

In [ ]:
%%bash -s "$JANO_FULL" "$WORKING_DATASET"

## Extract best blast hit for each gene and taxon
cut -f 1 $1 | grep -v "gene" | sort -u | while read line; do
    for i in "$2"/"$line".fasta__*; do
        cat $i | head -n 1 | cut -f2 >> "$2"/"$line"_toAdd.list
    done
done

In [ ]:
%%bash -s "$GENES_ADD" "$WORKING_DATASET"

## Extract best blast hit for each gene and taxon
cat $1 | while read line; do
    for i in "$2"/"$line".fasta__*; do
        cat $i | head -n 1 | cut -f2 >> "$2"/"$line"_toAdd.list
    done
done

I manually went through each {gene}_toAdd.list file and removed the hits that were from other genes!!! 

We pull the corresponding sequences now. 

First we concatenate all the proteomes together to make it easier to search the sequences.

In [ ]:
%%bash -s "$PROTEOMES" "$TO_ADD" 
for i in "$1"/*fasta; do (cat "${i}"; echo) >> "$2"/all.fasta; done

Next we use [seqkit](https://github.com/shenwei356/seqkit) to pull out the sequences for each gene.

In [ ]:
%%bash -s "$WORKING_DATASET" "$TO_ADD"

for i in "$1"/*_toAdd.list; do
    gene=$(basename $i | cut -f 1 -d '.')
    seqkit grep -f $i "$2"/all.fasta > "$1"/"$gene".fasta
done

Put together the query and the extracted sequences. 

In [ ]:
%%bash -s "$JANO_FULL" "$WORKING_DATASET"

cut -f 1 $1 | grep -v "gene" | sort -u | while read line; do
    cat "$2"/"$line".fasta "$2"/"$line"_toAdd.fasta > "$2"/"$line".all.fasta
done

In [ ]:
%%bash -s "$GENES_ADD" "$WORKING_DATASET"

cat $1 | while read line; do
    cat "$2"/"$line".fasta "$2"/"$line"_toAdd.fasta > "$2"/"$line".all.fasta
done

Before running the trees, we probably need to format the fasta headers so that the tree inference programme is happy. That means replacing ';' and '.' with '-'. 

We also edited our fasta headers so they would work with PhyloFisher (which we were originally planning to use for parsing our gene trees - however, we ended up using FigTree at the end). Since PhyloFisher expects the fasta headers to be identical across all genes for each taxon, all extra information apart from the taxon name should be after an underscore (as PhyloFisher seems to ignore everything after the underscore).

Finally we also use seqkit to rename any duplicates that may exist.

In [ ]:
# Folder containing output fasta files
DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['ROOT']

In [ ]:
%%bash -s "$WORKING_DATASET" "$DATASET"

for i in "$1"/*all.fasta; do
    gene=$(basename $i | cut -f 1 -d '.')
    cat $i | \
    tr '_' '-' | \
    tr ';' '-' | \
    sed -E 's/>(gene=.*-)(mag=.*)-(contig=.*)/>\2_\1\3/' | \
    seqkit rename \
    > "$2"/"$gene".fasta
done

### 1.5 Taxonomically label MAGs

Here, we use the taxonomy derived from a preliminary concatenated tree of 34 genes to label each MAG. 

In [ ]:
# Folder containing output fasta files
DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['ROOT']

In [ ]:
%%bash -s "$DATASET"

for i in "$1"/*fasta
    do cat $i | \
    tr '_' '\t' | \
    seqkit replace -p '^(\S+)(.+?)$' -r '{kv}$2' -k "$1"/mags_taxonomy.tsv --keep-key | \
    tr '\t' '_' \
    > tmp
    mv tmp $i
done

## 2 Align, trim, and infer single gene trees



### 2.1 Align

We align the genes with mafft-ginsi using the --unalign 0.6 option to avoid over-alignment.

In [12]:
from gene_iterator import GeneIterator

In [ ]:
# Folder with extracted gene dataset
DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['ROOT']

# Read_genes
JANO_FULL = paths_dict['DATABASES']['GENE_LISTS']['JANO_FULL']
genes = set(line.split()[0].strip() for line in open(JANO_FULL, "r"))

# Read genes
GENES_ADD = paths_dict['DATABASES']["GENE_LISTS"]["GENES_ADD"]
genes_add = set(line.split()[0].strip() for line in open(GENES_ADD, "r"))

# Directory for mafft output
MAFFT_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['MAFFT']

# Slurmlog csv
SLURMLOG = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['MAFFTLOG']


In [ ]:
gi = GeneIterator(DATASET, gene_list=genes, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi = GeneIterator(DATASET, gene_list=genes_add, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi.run_mafft(MAFFT_DIR, SLURMLOG)

How long are the alignments for each gene?

In [ ]:
%%bash -s "$MAFFT_DIR"

seqkit stats "$1"/*fasta

### 2.3 Trim

A number of studies (e.g. [Tan et al 2015](https://doi.org/10.1093/sysbio/syv033), [Irisarri et al 2022](https://doi.org/10.1093/sysbio/syab036)) have shown that aggressive trimming gives worse results. We therefore opted for gentle trimming with TrimAl by removing columns with 90% or more gaps. 

In [ ]:
# Folder containing aligned fasta files
MAFFT_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['MAFFT']

# Read_genes
JANO_FULL = paths_dict['DATABASES']['GENE_LISTS']['JANO_FULL']
genes = set(line.split()[0].strip() for line in open(JANO_FULL, "r"))

# Read genes
GENES_ADD = paths_dict['DATABASES']["GENE_LISTS"]["GENES_ADD"]
genes_add = set(line.split()[0].strip() for line in open(GENES_ADD, "r"))

# Directory for trimal output
TRIMAL_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['TRIMAL']

# Slurmlog csv
SLURMLOG = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['TRIMALLOG']

In [ ]:
gi = GeneIterator(MAFFT_DIR, gene_list=genes, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi = GeneIterator(MAFFT_DIR, gene_list=genes_add, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi.run_trimal(TRIMAL_DIR, SLURMLOG, MAFFT_DIR)

How long are the alignments for each gene?

In [ ]:
%%bash -s "$TRIMAL_DIR"

seqkit stats "$1"/*fasta

### 2.4 Infer trees with IQ-TREE

We infer trees with [IQ-TREE](http://www.iqtree.org/) using the best fitting model for each gene. 

In [13]:
# Folder containing aligned, trimmed fasta files
TRIMAL_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['ALIGNMENTS']['REF_MAGS']['TRIMAL']

# Read_genes
JANO_FULL = paths_dict['DATABASES']['GENE_LISTS']['JANO_FULL']
genes = set(line.split()[0].strip() for line in open(JANO_FULL, "r"))

# Read genes
GENES_ADD = paths_dict['DATABASES']["GENE_LISTS"]["GENES_ADD"]
genes_add = set(line.split()[0].strip() for line in open(GENES_ADD, "r"))

# Directory for trees output
TREES_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['TREES']['REF_MAGS']

# Slurmlog csv
SLURMLOG = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['TREES']['TREESLOG']

In [ ]:
gi = GeneIterator(TRIMAL_DIR, gene_list=genes, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi = GeneIterator(TRIMAL_DIR, gene_list=genes_add, suffix="fasta")
gi.unlock_pipeline()

In [ ]:
gi.run_siqtree(TREES_DIR, SLURMLOG, TRIMAL_DIR)

## 3. Parse Single Gene Trees
We manually parsed the trees to remove duplicates and contaminants. To help us do so, we colour the tips corresponding to references and duplicates (in FigTree). 

The following colour scheme is used:  
References = blue (#0000ff)  
Red-plastid-lineage MAGs = orange (#ff8000)  
Green-plastid-lineage MAGs = green (#00ff00)  
Duplicates = red (#ff0000)  
Other 'weird' artefacts (super long branches etc) = red (#ff0000)  
Taxa filtered in the previous step = cyan (#00ffff)  
Bacterial contaminants = purple (#800080) 

Changes are documented in the text file  `Notes.txt` (intermediate file, not provided).

## 4. Clean dataset

We can now remove artefacts and duplicates from our dataset. 

### 4.1 Extract bacterial contaminants (also referred to as paralogs here)

We start by extracting all the sequences marked as bacterial origin (in purple). It is important to retain these separately as 'paralogs' so that bacterial contaminants can be easily detected if we add more data. 

In [ ]:
# Folder containing the original dataset as fasta files
DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['ROOT']

# Folder containing the parsed trees
TREES_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['TREES']['REF_MAGS']

# Output folder for paralog fasta files 
PARALOGS = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['FINAL']['PARALOGS']

In [ ]:
%%bash -s "$TREES_DIR" "$DATASET" "$PARALOGS"

## For each parsed tree file, get the list of bacterial seqs
for i in "$1"/*parsed.tre
do
    gene=$(basename $i | cut -f 1 -d '.')
    grep "#800080" $i | \
    grep "mag_" | \
    sed -E 's/\s+(.*)\[\&\!color=#800080\]/\1/' | \
    sed -E 's/mag_/mag=/' | \
    sed -E 's/taxonomy_/taxonomy=/' | \
    sed -E 's/gene_/gene=/' | \
    sed -E 's/contig_/contig=/' | \
    sed -E 's/Filtered_/Filtered*/g' \
    > "$1"/paralogs.list
## extract the seqs from the original dataset and output to paralog file
    seqkit grep -f "$1"/paralogs.list "$2"/"$gene".fasta > "$3"/"$gene"_paralogs.fasta
done


We edit the fasta headers to add the word "paralog".

In [ ]:
%%bash -s "$PARALOGS"

for i in "$1"/*fasta
do 
    cat $i | \
    sed -E 's/(>.*)/\1_paralog/' \
    > fasta
    mv fasta $i
done

### 4.2 Extract clean sequences

We now extract the clean sequences.

In [ ]:
# Folder containing the original dataset as fasta files
DATASET = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['WORKING']['ROOT']

# Folder containing the parsed trees
TREES_DIR = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['TREES']['REF_MAGS']

# Output folder for clean, final, fasta files 
CLEAN = paths_dict['ANALYSIS_DATA']['SINGLE_GENE_ANALYSIS']['DATASET']['REF_MAGS']['FINAL']['CLEAN']

For each gene, we remove the artefacts and duplicates. 

In [ ]:
%%bash -s "$TREES_DIR"

## For each parsed tree file, get the list of artefacts and duplicates.
for i in "$1"/*parsed.tre
do
    gene=$(basename $i | cut -f 1 -d '.')
    echo $gene
    egrep "#800080|#ff0000|#00ffff" $i | \
    sed -E 's/\s+(.*)\[\&\!color=.*\]/\1/' | \
    sed -E 's/mag_/mag=/' | \
    sed -E 's/taxonomy_/taxonomy=/' | \
    sed -E 's/gene_/gene=/' | \
    sed -E 's/contig_/contig=/' | \
    sed -E 's/Filtered_/Filtered*/g' | \
    sed -E 's/taxo_/taxo=/' | \
    sed -E 's/accession_/accession=/' \
    > "$1"/"$gene".remove.list
    wc -l "$1"/"$gene".remove.list
done

Remove the following from each remove.list file as I had a made a mistake when assigning taxonomy.
 
CHL-AON-Bin-218-11-c  
CHL-AOS-Bin-120-3-c (remove only duplicate from psaA)  
CHL-MED-Bin-373-2-c  

In [ ]:
%%bash -s "$TREES_DIR" "$DATASET" "$CLEAN"

for i in "$1"/*remove.list
do 
    gene=$(basename $i | cut -f 1 -d '.')
    echo $gene
    seqkit grep -f $i "$2"/"$gene".fasta -v > "$3"/"$gene".fasta
done 

We check to see how many taxa are marked multiple times. Taxa marked more than once are likely chimeras/artefacts, and should therefore be removed from the whole dataset.

In [ ]:
%%bash -s "$TREES_DIR" 

cat "$1"/*remove.list | \
grep "mag=" | \
cut -f1 -d '_' | \
sort | \
uniq -c | \
grep " 1 mag" -v | \
sed -E 's/.*(mag.*)/\1/' \
> "$1"/all.remove.list

97 taxa were marked multiple times. I manually checked each of these 97 taxa and looked at *why* they were marked. A few simply included in this list when I marked duplicates, and some had a long branches in a couple of gene trees, but otherwise okay. I therefore removed the following from `all.remove.list`.

CHL-ARC-Bin-238-20-c  
CHL-AOS-Bin-109-4-c  
CHL-AOS-Bin-120-3-c  
CHL-AON-Bin-218-11-c  
CHL-AON-Bin-136-6-c  
CHL-AON-Bin-136-10-c  
CHL-ARC-Bin-238-31-c  
CHL-ARC-Bin-270-17-c  
CHL-ION-Bin-101-4-c  
CHL-ION-Bin-84-8-c  
CHL-MED-Bin-250-48-c  
CHL-MED-Bin-373-2-c  
CHL-PON-Bin-64-17-c  
CHL-PSW-Bin-163-25-c  
CHL-PSW-Bin-61-3-c  
CHL-PSW-Bin-87-1-c  
TARA-B110000977-METAG-scaffold-147  
CHL-SOC-Bin-154-29-c  
CHL-ARC-Bin-318-2-c  
CHL-MED-Bin-250-53-c  
CHL-PON-Bin-102-2-c  
CHL-PSW-Bin-50-1-c  
CHL-SOC-Bin-140-6-c  
CHL-SOC-Bin-154-29-c
CHL-AON-Bin-218-24-c
CHL-SOC-Bin-140-6-c

After removing, I was left with 83 taxa to remove.

In [ ]:
%%bash -s "$TREES_DIR" "$CLEAN"

for i in "$2"/*fasta
do 
    seqkit grep -r -f "$1"/all.remove.list $i -v > fasta
    mv fasta $i
done 

### 4.3 Rename sequences

Finally we rename the sequences so that taxa share the exact same headers across different genes. 

In [ ]:
%%bash -s "$CLEAN"

for i in "$1"/*fasta
do 
    cat $i | cut -f1 -d '_' > fasta
    mv fasta $i
done 

## References

Ševčíková, T., Horák, A., Klimeš, V., Zbránková, V., Demir-Hilton, E., Sudek, S., ... & Eliáš, M. (2015). Updating algal evolutionary relationships through plastid genome sequencing: did alveolate plastids emerge through endosymbiosis of an ochrophyte?. Scientific reports, 5(1), 10134. https://doi.org/10.1038/srep10134

Janouškovec, J., Horák, A., Oborník, M., Lukeš, J., & Keeling, P. J. (2010). A common red algal origin of the apicomplexan, dinoflagellate, and heterokont plastids. Proceedings of the National Academy of Sciences, 107(24), 10949-10954. https://doi.org/10.1073/pnas.1003335107

Shen, W., Le, S., Li, Y., & Hu, F. (2016). SeqKit: a cross-platform and ultrafast toolkit for FASTA/Q file manipulation. PloS one, 11(10), e0163962. https://doi.org/10.1371/journal.pone.0163962

Rambaut, A. (2009). FigTree. Tree figure drawing tool. http://tree.bio.ed.ac.uk/software/figtree/.

Tice, A. K., Žihala, D., Pánek, T., Jones, R. E., Salomaki, E. D., Nenarokov, S., ... & Brown, M. W. (2021). PhyloFisher: a phylogenomic package for resolving eukaryotic relationships. PLoS Biology, 19(8), e3001365. https://doi.org/10.1371/journal.pbio.3001365